In [1]:
from wand.image import Image as WImage
from wand.color import Color
from wand.api import library
from PIL import Image as PI
import io
import os
import pyocr
import pyocr.builders
import pandas as pd
import re
import os
from datamatch import (
    JaroWinklerSimilarity,
    ThresholdMatcher,
    ColumnsIndex,
)

In [2]:
# def ocr_pdfs():
#     pyocr.tesseract.TESSERACT_CMD = r"C:/Program Files/Tesseract-OCR/tesseract.exe"
#     dir_path = os.chdir("C:/Users/PC/Desktop/keepers/ok/data/CT/pdfs")
#     for file in os.listdir(dir_path):
#         if file.endswith(".pdf"):
#             tools = pyocr.get_available_tools()
#             if len(tools) == 0:
#                 raise Exception("No tool available")

#             tool = tools[0]
#             print("Will use tool '%s'" % (tool.get_name()))
#             langs = tool.get_available_languages()
#             print("Available languages: %s" % ", ".join(langs))
#             lang = langs[0]  # For English
#             print("Will use language '%s'" % (lang))

#             req_image = []
#             final_text = {}

#             with WImage(filename=file, resolution=300) as image_pdf:
#                 image_jpeg = image_pdf.convert("pdf")

#             try:
#                 for img in image_jpeg.sequence:
#                     img_page = WImage(image=img)
#                     req_image.append(img_page.make_blob("jpeg"))
#             finally:
#                 image_jpeg.destroy()

#             i = 0
#             for img in req_image:
#                 txt = tool.image_to_string(
#                     PI.open(io.BytesIO(img)),
#                     lang=lang,
#                     builder=pyocr.builders.TextBuilder(),
#                 )
#                 final_text[str(i)] = txt
#                 i += 1

#             text = final_text.items()
#             df = pd.DataFrame(text)
#             file_name = file + ".txt"
#             path = r"C:/Users/PC/Desktop/keepers/ok/data/CT/ocr/"
#             df.to_csv(path + file_name)
#     return df

In [3]:
# write = ocr_pdfs()

In [4]:
# to join OCR'd txt files: 1) cd into directory, 2) enter the command 'copy *.txt new_haven_allegations(or any filename).txt'

In [7]:
import os
os.getcwd()

'c:\\Users\\PC\\Desktop\\keepers\\ok\\notebooks'

,rank_desc,first_name,last_name,badge_no,officers
0,NaN,internal,affairs,division (oic),Internal Affairs Division (OIC)
1,NaN,l,affairs,unit,L AFFAIRS UNIT
2,NaN,internal,affairs,division (oic),Internal Affairs Division (OIC)
3,NaN,l,affairs,unit,L AFFAIRS UNIT
4,NaN,internal,affairs,division (oic),Internal Affairs Division (OIC)
...,...,...,...,...,...
13410,NaN,the,subject,he with was,the subject he with was.
13411,NaN,did,not,find any body-,did not find any body-
13412,NaN,iolation,of,body-worn camera general order,iolation of Body-worn camera General order.
13413,NaN,e,disposition,:,e Disposition:


In [15]:
# df.to_csv("../data/CT/ct_misconduct.csv")

In [16]:
# search = df[df.officers.str.contains("Vernik")]
# search